# When does visibility of artifacts and collections change?

Any record in LaminDB has the following 3 levels of visibility:
- 1: "default", visible by default
- 0: "archive", excluded from query & search by default
- -1: "trash", excluded from the query results, set with `.delete()`

These values are represented in the database via an integer field `_branch_code` that also models the branches involved in merge requests.

In [ ]:
# !pip install lamindb
!lamin init --storage test-_branch_code

In [ ]:
import lamindb as ln
import pandas as pd

In [ ]:
artifact = ln.Artifact.from_df(
    pd.DataFrame({"a": [1, 2], "b": [3, 4]}), description="mydf"
)
artifact.save()

New artifact has default _branch_code 1:

In [ ]:
assert artifact._branch_code == 1

When you delete an artifact, its _branch_code is set to -1 ("trash"):

In [ ]:
artifact.delete()

In [ ]:
assert artifact._branch_code == -1

Files in trash won't be returned from default queries:

In [ ]:
ln.Artifact.filter(description="mydf").all()

Unless you specify `_branch_code=None` to see all hidden and trashed artifacts (note that this only works because `_branch_code` is never `None`; fields that can be `None` would return records where the field is set to `None`).

In [ ]:
ln.Artifact.filter(description="mydf", _branch_code=None).all()

You can restore an artifact from trash:

In [ ]:
artifact.restore()

In [ ]:
assert artifact._branch_code == 1

In [ ]:
ln.Artifact.filter(description="mydf").all()

Delete test artifact and instance:

In [ ]:
artifact.delete(permanent=True)

In [ ]:
!lamin delete --force test-_branch_code